In [ ]:
import numpy as np

def generate_problem_parameters(n: int, seed: int = 42) -> tuple:
    """
    Generates correlated joint prior, cost vector, and miss-detection rates
    for a two-object search problem.

    Args:
        n (int): The number of cells.
        seed (int): Seed for reproducibility.

    Returns:
        tuple: (prior_joint, gamma1, gamma2, c)
    """
    np.random.seed(seed)

    # 1. Generate gamma1 in a range that allows gamma2 to be smaller
    gamma1 = np.random.uniform(low=0.6, high=0.85, size=n)

    # 2. Generate gamma2 such that gamma2 < gamma1, and within its own desired range [0.15, 0.25]
    gamma2 = np.zeros(n)
    min_gamma2_val = 0.15
    max_gamma2_val = 0.25
    for i in range(n):
        # Upper bound for gamma2 is min(max_gamma2_val, gamma1[i] - 0.02)
        upper_bound_for_this_gamma2 = min(max_gamma2_val, gamma1[i] - 0.02) # Small buffer to ensure gamma2 < gamma1
        if upper_bound_for_this_gamma2 < min_gamma2_val: # If buffer makes it too small, ensure min
            gamma2[i] = np.random.uniform(low=min_gamma2_val, high=min_gamma2_val + 0.01) # Force a small valid range
        else:
            gamma2[i] = np.random.uniform(low=min_gamma2_val, high=upper_bound_for_this_gamma2)
    gamma2 = np.clip(gamma2, None, max_gamma2_val) # Ensure max bound

    # 3. Generate cost vector c
    c = np.random.uniform(low=0.1, high=0.3, size=n)

    # 4. Generate correlated joint prior (n x n matrix)
    prior_joint = np.zeros((n, n))
    correlation_strength = 2.0 # Adjust this value to control correlation strength

    for i in range(n):
        for j in range(n):
            # Higher probability when O1 and O2 are in the same or adjacent cells
            distance = abs(i - j)
            prior_joint[i, j] = np.exp(-correlation_strength * distance) # Exponential decay with distance

    # Add some random noise to ensure variety (but keep it small)
    prior_joint += np.random.uniform(low=0.01, high=0.05, size=(n, n))
    prior_joint[prior_joint < 0.001] = 0.001 # Ensure no zero probabilities

    # Normalize the joint prior
    prior_joint /= np.sum(prior_joint)

    # Explicitly ensure the sum is exactly 1.0 by adjusting the last element
    # This handles potential floating-point inaccuracies after normalization.
    prior_joint[-1, -1] += (1.0 - np.sum(prior_joint))

    return prior_joint, gamma1, gamma2, c

if __name__ == '__main__':
    # Example usage:
    """NUM_CELLS = 25
    SEED = 20

    prior_joint, gamma1, gamma2, c = generate_problem_parameters(NUM_CELLS, SEED)

    print(f"Generated Parameters (N={NUM_CELLS}, Seed={SEED}):\n")
    print("Joint Prior (p(O1, O2)):\n", np.round(prior_joint, 4))
    print("\nSum of Joint Prior: ", np.sum(prior_joint))
    print("\nMiss-detection rates for O1 (gamma1):", np.round(gamma1, 4))
    print("Miss-detection rates for O2 (gamma2):", np.round(gamma2, 4))
    print("\nCost vector (c):", np.round(c, 4))

    # Verify gamma1 > gamma2
    print("\nIs gamma1 > gamma2 for all elements?:", np.all(gamma1 > gamma2))"""
    print(" ")

In [3]:
import numpy as np
from functools import lru_cache
from typing import Dict, List, Tuple, Any
import time

# --- Start of SingleObjectDPSolver (copied from txTdQaQVhkrA) ---
class SingleObjectDPSolver:
    """
    An exact Dynamic Programming solver for a single-object search problem.

    This class uses backward recursion to find the optimal policy based on a
    marginal prior distribution for a single target.
    """

    def __init__(self, n: int, T: int, p0_marginal: np.ndarray, gamma1: np.ndarray, c: np.ndarray):
        """
        Initializes the solver with the single-object problem parameters.

        Args:
            n (int): The number of cells.
            T (int): The time horizon.
            p0_marginal (np.ndarray): The n-element marginal prior probability vector for O1.
            gamma1 (np.ndarray): Miss-detection rates for O1.
        """
        self.n = n
        self.T = T
        self.p0_marginal = p0_marginal
        self.gamma1 = gamma1
        self.c = c

        # Data structures to store the results
        self.J_values: Dict[int, Dict[Tuple[int, ...], float]] = {}  # Value function J_t(z_t)
        self.Policy: Dict[int, Dict[Tuple[int, ...], int]] = {}      # Policy mu_t(z_t)

    @staticmethod
    @lru_cache(maxsize=None) # Memoization for performance
    def _generate_z_vectors(t: int, n: int) -> List[Tuple[int, ...]]:
        """Recursively generates all state vectors z_t where sum(z_i) = t."""
        if n == 1:
            return [(t,)]

        vectors = []
        for i in range(t + 1):
            for sub_vector in SingleObjectDPSolver._generate_z_vectors(t - i, n - 1):
                vectors.append((i,) + sub_vector)
        return vectors

    def _calculate_posterior(self, z_vector: np.ndarray) -> np.ndarray:
        """Calculates the posterior belief p(O1=i | z)."""
        g1_z = np.power(self.gamma1, z_vector)
        numerator = self.p0_marginal * g1_z
        norm = np.sum(numerator)
        return numerator / norm if norm > 0 else numerator

    def solve(self):
        """
        Executes the backward recursion to solve the DP problem.
        """
        # print("Starting Single-Object DP solver...")
        # --- Initialization at T ---
        # print(f"Initializing for T={self.T}...")
        self.J_values[self.T] = {}
        z_vectors_T = self._generate_z_vectors(self.T, self.n)
        for z_T in z_vectors_T:
            self.J_values[self.T][z_T] = 0.0

        # --- Backward Recursion ---
        for t in range(self.T - 1, -1, -1):
            #start_time = time.time()
            J_t = {}
            policy_t = {}
            z_vectors_t = self._generate_z_vectors(t, self.n)

            for z_t_tuple in z_vectors_t:
                z_t = np.array(z_t_tuple)
                action_values = []

                # Calculate current belief vector
                belief_t = self._calculate_posterior(z_t)

                # Iterate over all possible actions
                for a_t in range(self.n):
                    # Probability of finding the target if we search cell a_t
                    p_success = (1 - self.gamma1[a_t]) * belief_t[a_t]
                    p_fail = 1 - ((1 - self.gamma1[a_t]) * belief_t[a_t])

                    # Get the value of the state we transition to upon failure
                    next_z_tuple = tuple(z_t + np.eye(self.n, dtype=int)[a_t])
                    val_if_fail = self.J_values[t + 1][next_z_tuple]

                    # Expected value for this action (Bellman equation)
                    # Reward for success is 1, reward for failure is 0
                    expected_value = (p_success * 1.0) - self.c[a_t] + (p_fail * val_if_fail)
                    action_values.append(expected_value)

                # Find best action and store value/policy
                best_value = np.max(action_values)
                best_action = np.argmax(action_values)
                J_t[z_t_tuple] = best_value
                policy_t[z_t_tuple] = best_action

            self.J_values[t] = J_t
            self.Policy[t] = policy_t
            #end_time = time.time()
            # print(f"Completed t={t}. Found {len(z_vectors_t)} states. Took {end_time - start_time:.2f}s.")

        # print("DP solver finished.")

    def get_optimal_value(self) -> float:
        """Returns the optimal value J(z_0)."""
        initial_z = tuple([0] * self.n)
        return self.J_values[0][initial_z]
# --- End of SingleObjectDPSolver ---


# --- Start of TwoObjectDPSolver (copied from Uu36DFAN5dbB) ---
class TwoObjectDPSolver:
    """
    An exact Dynamic Programming solver for the two-object search problem.

    This class uses backward recursion to find the optimal policy and the
    maximum probability of detecting the primary target (O1).
    """

    def __init__(self, n: int, T: int, p0: np.ndarray, gamma1: np.ndarray, gamma2: np.ndarray, c: np.ndarray):
        """
        Initializes the solver with the problem parameters.

        Args:
            n (int): The number of cells.
            T (int): The time horizon.
            p0 (np.ndarray): The n x n prior joint probability matrix.
            gamma1 (np.ndarray): Miss-detection rates for O1.
            gamma2 (np.ndarray): Miss-detection rates for O2.
        """
        self.n = n
        self.T = T
        self.p0 = p0
        self.gamma1 = gamma1
        self.gamma2 = gamma2
        self.c = c

        # Data structures to store the results
        self.J_values: Dict[int, Dict[Tuple, float]] = {}  # Value function J_t(s_t)
        self.Policy: Dict[int, Dict[Tuple, int]] = {}      # Policy mu_t(s_t)

        # Pre-calculate initial conditional priors for efficiency
        self._p0_conditionals = self._precompute_p0_conditionals()

    def _precompute_p0_conditionals(self) -> List[np.ndarray]:
        """Pre-calculates P(O1=i | O2=k) for all k."""
        conditionals = []
        for k in range(self.n):
            marginal_o2 = np.sum(self.p0[:, k])
            if marginal_o2 > 0:
                conditionals.append(self.p0[:, k] / marginal_o2)
            else:
                # If O2 can never be in cell k, the conditional is undefined. Use zeros.
                conditionals.append(np.zeros(self.n))
        return conditionals

    @staticmethod
    @lru_cache(maxsize=None) # Memoization for performance
    def _generate_z_vectors(t: int, n: int) -> List[Tuple[int, ...]]:
        """Recursively generates all state vectors z_t where sum(z_i) = t."""
        if n == 1:
            return [(t,)]

        vectors = []
        for i in range(t + 1):
            for sub_vector in TwoObjectDPSolver._generate_z_vectors(t - i, n - 1):
                vectors.append((i,) + sub_vector)
        """if n == 5:
           for vec in vectors:
               print(vec)"""
        return vectors

    def _calculate_joint_posterior(self, z_vector: np.ndarray) -> np.ndarray:
        """Calculates p(O1=i, O2=j | z)."""
        g1_z = np.power(self.gamma1, z_vector)
        g2_z = np.power(self.gamma2, z_vector)
        likelihood = np.outer(g1_z, g2_z)
        numerator = self.p0 * likelihood
        norm = np.sum(numerator)
        return numerator / norm if norm > 0 else numerator

    def _calculate_conditional_posterior(self, z_vector: np.ndarray, o2_loc: int) -> np.ndarray:
        """Calculates p(O1=i | z, O2=o2_loc)."""
        p0_cond = self._p0_conditionals[o2_loc]
        g1_z = np.power(self.gamma1, z_vector)
        numerator = g1_z * p0_cond
        norm = np.sum(numerator)
        return numerator / norm if norm > 0 else numerator

    def solve(self):
        """
        Executes the backward recursion to solve the DP problem.
        """
        #print("Starting DP solver...")
        # --- Initialization at T ---
        #print(f"Initializing for T={self.T}...")
        self.J_values[self.T] = {}
        z_vectors_T = self._generate_z_vectors(self.T, self.n)
        for z_T in z_vectors_T:
            for theta2 in range(self.n + 1):
                state = (z_T, theta2)
                self.J_values[self.T][state] = 0.0

        # --- Backward Recursion ---
        for t in range(self.T - 1, -1, -1):
            #start_time = time.time()
            J_t = {}
            policy_t = {}
            z_vectors_t = self._generate_z_vectors(t, self.n)

            for z_t_tuple in z_vectors_t:
                z_t = np.array(z_t_tuple)
                for theta2 in range(self.n + 1):
                    if t == 0 and theta2 > 0:
                        break
                    current_state = (z_t_tuple, theta2)
                    action_values = []

                    # Calculate belief based on current state
                    if theta2 == 0:
                        belief = self._calculate_joint_posterior(z_t)
                    else:
                        o2_loc = theta2 - 1
                        belief = self._calculate_conditional_posterior(z_t, o2_loc)

                    # Iterate over all possible actions
                    for a_t in range(self.n):
                        next_z_tuple = tuple(z_t + np.eye(self.n, dtype=int)[a_t])

                        # --- Case 1: Both objects hidden ---
                        if theta2 == 0:
                            p_marginal_o1_at_a = np.sum(belief[a_t, :])

                            # Prob of success (finding O1)
                            p_success = (1 - self.gamma1[a_t]) * p_marginal_o1_at_a

                            # Prob of finding O2 only
                            p_cond_sum = np.sum(belief[np.arange(self.n) != a_t, a_t])
                            p_find_o2_only = (1 - self.gamma2[a_t]) * (self.gamma1[a_t] * belief[a_t, a_t] + p_cond_sum)

                            # Prob of finding nothing
                            p_nothing = 1 - p_success - p_find_o2_only

                            # Future values from next stage
                            val_if_nothing = self.J_values[t + 1][(next_z_tuple, 0)]
                            val_if_found_o2 = self.J_values[t + 1][(next_z_tuple, a_t + 1)]

                            # Expected value for this action
                            expected_value = (p_success * 1.0) - self.c[a_t]+ \
                                             (p_nothing * val_if_nothing) + \
                                             (p_find_o2_only * val_if_found_o2)
                            action_values.append(expected_value)

                        # --- Case 2: O2 has been found ---
                        else:
                            p_o1_at_a = belief[a_t]
                            p_success = (1 - self.gamma1[a_t]) * p_o1_at_a
                            p_fail = 1 - p_success

                            val_if_fail = self.J_values[t + 1][(next_z_tuple, theta2)]

                            expected_value = (p_success * 1.0) - self.c[a_t]+ (p_fail * val_if_fail)
                            action_values.append(expected_value)

                    # Find best action and store value/policy
                    best_value = np.max(action_values)
                    best_action = np.argmax(action_values)
                    J_t[current_state] = best_value
                    policy_t[current_state] = best_action

            self.J_values[t] = J_t
            self.Policy[t] = policy_t
            #end_time = time.time()
            #print(f"Completed t={t}. Found {len(z_vectors_t)} states. Took {end_time - start_time:.2f}s.")

        #print("DP solver finished.")

    def get_optimal_value(self) -> float:
        """Returns the optimal value J(s_0)."""
        initial_z = tuple([0] * self.n)
        initial_state = (initial_z, 0)
        return self.J_values[0][initial_state]
# --- End of TwoObjectDPSolver ---


class KStepLookaheadSolver:
    """
    A k-step lookahead solver using rollout with a greedy policy for value estimation.
    """

    def __init__(self, n: int, T: int, initial_episode_belief: Any, # Can be (n,n) or (n,)
                 gamma1: np.ndarray, gamma2: np.ndarray, theta2_at_init: int, k: int, c: np.ndarray, rollout_sims: int = 100, solver_seed: int = None):
        self.n = n
        self.T_global = T # Store global horizon for rollout remaining_horizon calculation
        self.initial_episode_belief = initial_episode_belief # This is the current belief from the global episode
        self.gamma1 = gamma1
        self.gamma2 = gamma2
        self.c = c
        self.k = k # Lookahead horizon for *this* solver instance
        self.rollout_sims = rollout_sims
        self.theta2_at_init = theta2_at_init # O2 state when this k-step solver was created

        self.J_values: Dict[int, Dict[Tuple, float]] = {}
        self.Policy: Dict[int, Dict[Tuple, int]] = {}

        # Initialize a seeded random number generator for reproducible rollouts
        self.rng = np.random.default_rng(solver_seed)

        # Precompute conditionals from the initial joint belief if theta2_at_init == 0
        # These are used if O2 is hidden initially, but discovered within the k-step lookahead
        if self.theta2_at_init == 0 and self.initial_episode_belief.ndim == 2:
            self._precomputed_conditionals_from_initial_joint = self._precompute_conditionals_from_joint(self.initial_episode_belief)
        else:
            self._precomputed_conditionals_from_initial_joint = None

    def _precompute_conditionals_from_joint(self, joint_p: np.ndarray) -> List[np.ndarray]:
        """Pre-calculates P(O1=i | O2=k) for all k from a given joint prior."""
        conditionals = []
        for k_idx in range(self.n):
            marginal_o2 = np.sum(joint_p[:, k_idx])
            conditionals.append(joint_p[:, k_idx] / marginal_o2 if marginal_o2 > 0 else np.zeros(self.n))
        return conditionals

    def _calculate_posterior_at_step_in_k_lookahead(self, z_relative: np.ndarray, current_theta2_in_k_step: int) -> Any:
        """
        Calculates the posterior belief for a state (z_relative, current_theta2_in_k_step)
        within this k-step lookahead segment, starting from `self.initial_episode_belief`.
        `z_relative`: observations accumulated *within this k-step window*.
        `current_theta2_in_k_step`: the theta2 state *within this k-step window*.
        """
        if current_theta2_in_k_step == 0: # O2 is still hidden
            # If self.theta2_at_init was already > 0, this case should not happen for a consistent path
            if self.initial_episode_belief.ndim == 1:
                # This implies an inconsistency, should raise error or handle carefully
                return np.zeros((self.n, self.n)) # Return a zero joint if initial was marginal

            g1_z = np.power(self.gamma1, z_relative)
            g2_z = np.power(self.gamma2, z_relative)
            likelihood = np.outer(g1_z, g2_z)
            numerator = self.initial_episode_belief * likelihood
            norm = np.sum(numerator)
            return numerator / norm if norm > 0 else numerator
        else: # O2 is found (either initially or within this k-step segment)
            if self.theta2_at_init > 0:
                # O2 was already found when KStepLookaheadSolver was instantiated
                # initial_episode_belief is already the marginal p(O1 | O2=actual_loc)
                base_prior_for_o1 = self.initial_episode_belief
            else:
                # O2 was hidden initially but found within the k-step lookahead
                # We need to get the conditional prior for O1 from the initial joint belief
                o2_loc = current_theta2_in_k_step - 1
                base_prior_for_o1 = self._precomputed_conditionals_from_initial_joint[o2_loc]

            g1_z = np.power(self.gamma1, z_relative)
            numerator = g1_z * base_prior_for_o1
            norm = np.sum(numerator)
            return numerator / norm if norm > 0 else numerator

    def _select_greedy_action(self, belief: np.ndarray, theta2_for_greedy: int) -> int:
        """Selects a greedy action based on the current belief."""
        if theta2_for_greedy == 0: # Belief is joint (n,n)
            p_marginal_o1 = np.sum(belief, axis=1)
            immediate_value = (1 - self.gamma1) * p_marginal_o1
        else: # Belief is marginal for O1 (n,)
            immediate_value = (1 - self.gamma1) * belief
        return np.argmax(immediate_value - self.c)


    def _run_rollout(self, rollout_z_relative: np.ndarray, rollout_theta2_in_k_step: int, remaining_global_horizon: int) -> float:
        """
        Estimates the value by simulating episodes with a greedy policy.
        `rollout_z_relative`: z_vector accumulated *within the k-step lookahead* before rollout.
        `rollout_theta2_in_k_step`: theta2 state *within the k-step lookahead* before rollout.
        `remaining_global_horizon`: Actual remaining steps in the *global* problem after the k-step lookahead.
        """
        total_reward = 0.0

        for _ in range(self.rollout_sims):
            current_z_in_rollout = np.copy(rollout_z_relative)
            current_theta2_in_rollout = rollout_theta2_in_k_step

            # Sample true object locations for this rollout simulation based on the *current belief*
            current_belief_for_sampling = self._calculate_posterior_at_step_in_k_lookahead(current_z_in_rollout, current_theta2_in_rollout)

            if current_theta2_in_rollout == 0: # Belief is joint
                flat_belief = current_belief_for_sampling.flatten()
                if np.sum(flat_belief) > 0:
                    choice_idx = self.rng.choice(self.n * self.n, p=flat_belief / np.sum(flat_belief))
                    rollout_true_pos_o1, rollout_true_pos_o2 = np.unravel_index(choice_idx, (self.n, self.n))
                else:
                    rollout_true_pos_o1, rollout_true_pos_o2 = -1, -1
            else: # Belief is marginal for O1
                if np.sum(current_belief_for_sampling) > 0:
                    rollout_true_pos_o1 = self.rng.choice(self.n, p=current_belief_for_sampling / np.sum(current_belief_for_sampling))
                    rollout_true_pos_o2 = current_theta2_in_rollout - 1 # O2 location is known
                else:
                    rollout_true_pos_o1, rollout_true_pos_o2 = -1, -1

            episode_rollout_reward = 0.0
            for t_rollout in range(remaining_global_horizon): # Iterate for remaining time steps
                # Get belief for greedy action decision (no z_relative update yet for this step's decision)
                current_belief_for_greedy_action = self._calculate_posterior_at_step_in_k_lookahead(current_z_in_rollout, current_theta2_in_rollout)
                action = self._select_greedy_action(current_belief_for_greedy_action, current_theta2_in_rollout)

                episode_rollout_reward -= self.c[action]

                # Simulate outcome of greedy action for rollout
                found_o1_in_rollout = (action == rollout_true_pos_o1) and (self.rng.random() > self.gamma1[action])
                if found_o1_in_rollout:
                    episode_rollout_reward += 1.0
                    break # O1 found, rollout ends

                if current_theta2_in_rollout == 0 and action == rollout_true_pos_o2 and (self.rng.random() > self.gamma2[action]):
                    current_theta2_in_rollout = action + 1 # O2 found within rollout

                current_z_in_rollout[action] += 1 # Update z for next rollout step
            total_reward += episode_rollout_reward
        return total_reward / self.rollout_sims

    @staticmethod
    @lru_cache(maxsize=None)
    def _generate_z_vectors(t: int, n: int) -> List[Tuple[int, ...]]:
        if n == 1:
            return [(t,)]
        vectors = []
        for i in range(t + 1):
            for sub_vector in KStepLookaheadSolver._generate_z_vectors(t - i, n - 1):
                vectors.append((i,) + sub_vector)
        return vectors

    def solve(self): # For theta2_at_init == 0
        """
        Executes DP for k steps from self.initial_episode_belief, for theta2_at_init = 0.
        """
        self.J_values[self.k] = {}
        z_vectors_k = self._generate_z_vectors(self.k, self.n)
        for z_k_relative_tuple in z_vectors_k:
            for theta2_in_k_step in range(self.n + 1):
                state = (z_k_relative_tuple, theta2_in_k_step)
                z_k_relative = np.array(z_k_relative_tuple)
                # Value at the leaf node of k-step DP is estimated by rollout
                self.J_values[self.k][state] = self._run_rollout(z_k_relative, theta2_in_k_step, self.T_global - self.k)


        for t_k_relative in range(self.k - 1, -1, -1):
            J_t = {}
            policy_t = {}
            z_vectors_t = self._generate_z_vectors(t_k_relative, self.n)

            for z_t_relative_tuple in z_vectors_t:
                for theta2_in_k_step in range(self.n + 1):
                    current_state_in_k_step = (z_t_relative_tuple, theta2_in_k_step)
                    action_values = []
                    z_t_relative = np.array(z_t_relative_tuple)

                    current_belief_in_k_step = self._calculate_posterior_at_step_in_k_lookahead(z_t_relative, theta2_in_k_step)

                    for action_k in range(self.n):
                        next_z_relative_tuple = tuple(z_t_relative + np.eye(self.n, dtype=int)[action_k])

                        if theta2_in_k_step == 0: # O2 still hidden in this k-step segment
                            # Current belief is joint (n,n)
                            p_marginal_o1_at_a = np.sum(current_belief_in_k_step[action_k, :])
                            p_success = (1 - self.gamma1[action_k]) * p_marginal_o1_at_a

                            # Prob of finding O2 only
                            p_find_o2_only = (1 - self.gamma2[action_k]) * (self.gamma1[action_k] * current_belief_in_k_step[action_k, action_k] + np.sum(current_belief_in_k_step[np.arange(self.n) != action_k, action_k]))

                            p_nothing = 1 - p_success - p_find_o2_only

                            val_if_nothing = self.J_values[t_k_relative + 1][(next_z_relative_tuple, 0)]
                            val_if_found_o2 = self.J_values[t_k_relative + 1][(next_z_relative_tuple, action_k + 1)]

                            expected_value = (p_success * 1.0) - self.c[action_k] + \
                                             (p_nothing * val_if_nothing) + \
                                             (p_find_o2_only * val_if_found_o2)
                            action_values.append(expected_value)

                        else: # O2 already found (theta2_in_k_step > 0)
                            # Current belief is marginal for O1 (n,)
                            p_o1_at_a = current_belief_in_k_step[action_k]
                            p_success = (1 - self.gamma1[action_k]) * p_o1_at_a
                            p_fail = 1 - p_success

                            val_if_fail = self.J_values[t_k_relative + 1][(next_z_relative_tuple, theta2_in_k_step)]

                            expected_value = (p_success * 1.0) - self.c[action_k] + (p_fail * val_if_fail)
                            action_values.append(expected_value)

                    best_value = np.max(action_values)
                    best_action = np.argmax(action_values)
                    J_t[current_state_in_k_step] = best_value
                    policy_t[current_state_in_k_step] = best_action

                self.J_values[t_k_relative] = J_t
                self.Policy[t_k_relative] = policy_t

    def solve_with_theta(self): # For theta2_at_init > 0
        """
        Executes DP for k steps from self.initial_episode_belief, for theta2_at_init > 0.
        In this case, O2 is already found, so theta2 is fixed throughout the k steps.
        """
        fixed_theta2_in_k_step = self.theta2_at_init

        self.J_values[self.k] = {}
        z_vectors_k = self._generate_z_vectors(self.k, self.n)
        for z_k_relative_tuple in z_vectors_k:
            z_k_relative = np.array(z_k_relative_tuple)
            state = (z_k_relative_tuple, fixed_theta2_in_k_step)
            # Value at the leaf node of k-step DP is estimated by rollout
            self.J_values[self.k][state] = self._run_rollout(z_k_relative, fixed_theta2_in_k_step, self.T_global - self.k)

        for t_k_relative in range(self.k - 1, -1, -1):
            J_t = {}
            policy_t = {}
            z_vectors_t = self._generate_z_vectors(t_k_relative, self.n)

            for z_t_relative_tuple in z_vectors_t:
                z_t_relative = np.array(z_t_relative_tuple)
                current_state_in_k_step = (z_t_relative_tuple, fixed_theta2_in_k_step)
                action_values = []

                current_belief_in_k_step = self._calculate_posterior_at_step_in_k_lookahead(z_t_relative, fixed_theta2_in_k_step)

                for action_k in range(self.n):
                    next_z_relative_tuple = tuple(z_t_relative + np.eye(self.n, dtype=int)[action_k])

                    p_o1_at_a = current_belief_in_k_step[action_k]
                    p_success = (1 - self.gamma1[action_k]) * p_o1_at_a
                    p_fail = 1 - p_success

                    val_if_fail = self.J_values[t_k_relative + 1][(next_z_relative_tuple, fixed_theta2_in_k_step)]

                    expected_value = (p_success * 1.0) - self.c[action_k] + (p_fail * val_if_fail)
                    action_values.append(expected_value)

                best_value = np.max(action_values)
                best_action = np.argmax(action_values)
                J_t[current_state_in_k_step] = best_value
                policy_t[current_state_in_k_step] = best_action

            self.J_values[t_k_relative] = J_t
            self.Policy[t_k_relative] = policy_t


# Helper function definitions (assuming they are available or defined above this class)
# Need to ensure calculate_joint_posterior and calculate_conditional_posterior are accessible.
# Let's define them here for clarity, or assume they are in a common helpers file.

# Re-including helper functions for self-containment
def calculate_joint_posterior(z_vector: np.ndarray, initial_prior_joint: np.ndarray,
                              gamma1: np.ndarray, gamma2: np.ndarray) -> np.ndarray:
    """Calculates the joint posterior belief p(O1=i, O2=j | z) based on initial joint prior."""
    g1_z = np.power(gamma1, z_vector)
    g2_z = np.power(gamma2, z_vector)
    likelihood = np.outer(g1_z, g2_z)
    numerator = initial_prior_joint * likelihood
    norm = np.sum(numerator)
    return numerator / norm if norm > 0 else numerator

def calculate_conditional_posterior(z_vector: np.ndarray, o2_loc: int,
                                    precomputed_conditionals: List[np.ndarray], gamma1: np.ndarray) -> np.ndarray:
    """Calculates the conditional posterior belief p(O1=i | z, O2=k) based on precomputed conditionals and current z_vector."""
    p0_conditional = precomputed_conditionals[o2_loc]
    g1_z = np.power(gamma1, z_vector)
    numerator = g1_z * p0_conditional
    norm = np.sum(numerator)
    return numerator / norm if norm > 0 else numerator

def precompute_p0_conditionals(n: int, p0_joint: np.ndarray) -> list:
    """Pre-calculates P(O1=i | O2=k) for all k from a joint prior matrix."""
    if p0_joint.ndim != 2: # Should be (n,n) for joint prior
        raise ValueError("p0 must be a 2D joint probability matrix for precomputing conditionals.")
    conditionals = []
    for k in range(n):
        marginal_o2 = np.sum(p0_joint[:, k])
        conditionals.append(p0_joint[:, k] / marginal_o2 if marginal_o2 > 0 else np.zeros(n))
    return conditionals



# =============================================================================
# 1. Adaptive Re-planning Episode Simulation
# =============================================================================

def run_k_step_episode(n: int, T: int, p0_initial_joint: np.ndarray,
                                    gamma1: np.ndarray, gamma2: np.ndarray, c: np.ndarray, k: int, ROLLOUT_SIMULATIONS: int, episode_seed: int) -> Tuple[bool, int, float, int]:
    """
    Simulates a single episode using k-step lookahead with greedy rollout and then full DP.
    Returns success status, time of detection (or -1 if failed), accumulated reward, and episode length.
    """
    # 1. Secretly determine the true locations for the entire episode
    flat_p0 = p0_initial_joint.flatten()
    if np.sum(flat_p0) == 0: # Handle case where prior is all zeros
         true_pos_o1, true_pos_o2 = -1, -1 # Indicate no target
    else:
        # Use np.random directly as its state is set by np.random.seed(episode_seed)
        choice_index = np.random.choice(n * n, p=flat_p0 / np.sum(flat_p0)) # Normalize
        true_pos_o1, true_pos_o2 = np.unravel_index(choice_index, (n, n))

    # 2. Initialize global state and reward variables
    z_vector_global = np.zeros(n, dtype=int) # Accumulated search history for the entire episode
    theta2_global = 0 # O2 state: 0=hidden, >0=found at cell (theta2_global-1)
    current_belief_global = np.copy(p0_initial_joint) # Evolving belief state (joint or marginal)

    # Precompute global conditionals from the initial joint prior once
    global_p0_conditionals = precompute_p0_conditionals(n, p0_initial_joint)

    accumulated_reward = 0.0
    detection_time = -1
    episode_length = 0

    # Phase 1: K-step lookahead for (T-k) time steps
    for t_global in range(T - k): # t_global is the current time step in the entire episode
        episode_length += 1

        # Create a unique, reproducible seed for this KStepLookaheadSolver instance
        solver_instance_seed = episode_seed * T + t_global

        # Instantiate K-step lookahead solver with the current global state (belief, z_vector, theta2)
        # The KStepLookaheadSolver will calculate policies for `k` steps starting from a relative z=0.
        solver_instance = KStepLookaheadSolver(n=n, T=T-t_global,
                                                initial_episode_belief=current_belief_global,
                                                gamma1=gamma1, gamma2=gamma2,
                                                theta2_at_init=theta2_global,
                                                k=k, c=c, rollout_sims=ROLLOUT_SIMULATIONS,
                                                solver_seed=solver_instance_seed)

        if theta2_global == 0: # If O2 is currently hidden, use the general solve method
            solver_instance.solve()
        else: # If O2 is currently found, use the solve method specialized for a fixed theta2
            solver_instance.solve_with_theta()

        # Get the optimal action for the current global step (t_global) from the k-step solver's policy
        # The k-step solver's policy is for its *relative* time t=0 and *relative* z=0.
        action = solver_instance.Policy[0][(tuple([0]*n), theta2_global)]

        # Deduct cost for the action
        accumulated_reward -= c[action]

        # Simulate outcome of the action
        found_o1 = (action == true_pos_o1) and (np.random.random() > gamma1[action])
        if found_o1:
            accumulated_reward += 1.0 # Reward for finding O1
            detection_time = t_global + 1 # Absolute time of detection
            return True, detection_time, accumulated_reward, episode_length # Mission Success!

        # Check if O2 is found *at this current global step* (only if it was previously hidden)
        if theta2_global == 0 and action == true_pos_o2 and (np.random.random() > gamma2[action]):
            theta2_global = action + 1 # O2 found at cell `action`, update state

        # Update global z_vector based on the action taken
        z_vector_global[action] += 1

        # Update global belief state based on the action and observed outcome
        if theta2_global == 0: # If O2 is still hidden globally
            current_belief_global = calculate_joint_posterior(z_vector_global, p0_initial_joint, gamma1, gamma2)
        else: # If O2 has been found globally
            current_belief_global = calculate_conditional_posterior(z_vector_global, theta2_global - 1, global_p0_conditionals, gamma1)

    # Phase 2: After T-k steps, switch to full DP for the remaining k steps
    # The remaining time horizon for this DP problem is `k`
    # The DP solvers operate on their own relative z_vector, taking `current_belief_global` as their starting prior.
    final_z_for_dp = np.zeros(n, dtype=int) # z_vector for the DP solver starts from 0 relative to its own horizon
    final_theta2_for_dp = theta2_global # O2 state is carried over to the DP phase
    final_policy = None

    if final_theta2_for_dp == 0: # O2 is still hidden, use TwoObjectDPSolver
        dp_solver = TwoObjectDPSolver(n=n, T=k, p0=current_belief_global, gamma1=gamma1, gamma2=gamma2, c=c)
        dp_solver.solve()
        final_policy = dp_solver.Policy
    else: # O2 has been found, use SingleObjectDPSolver
        dp_solver = SingleObjectDPSolver(n=n, T=k, p0_marginal=current_belief_global, gamma1=gamma1, c=c)
        dp_solver.solve()
        final_policy = dp_solver.Policy

    for t_dp in range(k): # Iterate for the remaining `k` time steps using the final DP policy
        episode_length += 1

        # Determine the state key for the DP policy based on theta2
        state_for_dp = (tuple(final_z_for_dp), final_theta2_for_dp) if final_theta2_for_dp == 0 else tuple(final_z_for_dp)

        if t_dp not in final_policy or state_for_dp not in final_policy[t_dp]:
            # This case should ideally not happen if the DP policy is complete for the horizon `k`
            # print(f"Warning: DP Policy not found for state {state_for_dp} at time {t_dp}. Ending episode.")
            return False, detection_time, accumulated_reward, episode_length

        action = final_policy[t_dp][state_for_dp]
        accumulated_reward -= c[action]

        # Simulate outcome of the action in the DP phase
        if action == true_pos_o1 and (np.random.random() > gamma1[action]):
            accumulated_reward += 1.0 # Reward for finding O1
            detection_time = (T - k) + (t_dp + 1) # Absolute time of detection
            return True, detection_time, accumulated_reward, episode_length # Mission Success!

        # Check for O2 discovery during DP phase, if O2 was still hidden
        if final_theta2_for_dp == 0 and action == true_pos_o2 and (np.random.random() > gamma2[action]):
            # If O2 is found here, the state `final_theta2_for_dp` is updated, but the DP policy
            # was computed assuming `final_theta2_for_dp` was `0` for the entire `k` steps.
            # A more advanced adaptive approach would re-plan here with a new SingleObjectDPSolver.
            # For this simplified model, we just update the state and continue with the same policy.
            final_theta2_for_dp = action + 1

        # Update z_vector for the DP solver's internal state
        final_z_for_dp[action] += 1

    return False, detection_time, accumulated_reward, episode_length # Mission Failed


# =============================================================================
# 2. MAIN EXPERIMENT SCRIPT (Modified for multiple seeds)
# =============================================================================

if __name__ == '__main__':
    # --- Problem Definition ---
    # Example usage for testing the greedy policy
    NUM_CELLS = 25
    TIME_HORIZON = 20
    NUM_EPISODES_PER_SEED = 10 # Number of episodes to run for this benchmark
    LOOKAHEAD_K = 1
    ROLLOUT_SIMULATIONS = 10


    print("--- Running Empirical Experiment for Greedy Policy ---")
    print(f"Number of episodes: {NUM_EPISODES}")
    print("-" * 20)


    SEED = 42

    prior, gammas1, gammas2, c = generate_problem_parameters(NUM_CELLS, SEED)

    ## Ensure reproducibility 
    assert (prior[0][0], prior[24][24]) == (0.020334334191265974, 0.020140323245187997)
    assert (gammas1[0], gammas1[1]) == (0.6936350297118405, 0.837678576602479)
    assert (gammas2[0], gammas2[1]) == (0.22851759613930137, 0.16996737821583596)
    assert (c[0], c[1]) ==( 0.2939169255529117, 0.2550265646722229)

    all_success_rates = []
    all_detection_times = []
    all_rewards = []
    all_episode_lengths = []
    total_experiment_start_time = time.time()

    print("--- Running Empirical Experiment for k-step lookahead Policy over Multiple Seeds ---")
    print(f"Number of seeds: {NUM_SEEDS}")
    print(f"Number of evaluation episodes per seed: {NUM_EPISODES_PER_SEED}")
    print(f"Lookahead depth (k): {LOOKAHEAD_K}")
    print(f"Rollout simulations: {ROLLOUT_SIMULATIONS}")
    print("-" * 20)


    for seed in range(NUM_SEEDS):
        print(f"\n--- Running with Seed: {seed} ---")
        np.random.seed(seed) # Set seed for main episode simulation

        num_successes = 0
        detection_times_this_seed = []
        rewards_this_seed = []
        episode_lengths_this_seed = []
        start_time_this_seed = time.time()

        for i in range(NUM_EPISODES_PER_SEED):
            # print(f"Episode : {i}") # Uncomment for detailed per-episode output
            success, detection_time, reward, episode_length = run_k_step_episode(NUM_CELLS, TIME_HORIZON, prior, gammas1, gammas2, c, LOOKAHEAD_K, ROLLOUT_SIMULATIONS, seed)
            rewards_this_seed.append(reward)
            episode_lengths_this_seed.append(episode_length)

            if success:
                num_successes += 1
                detection_times_this_seed.append(detection_time)
                # print(f"    (Episode {i+1} Result) SUCCESS at time {detection_time} with reward {reward:.4f}")
            # else:
                # print(f"    (Episode {i+1} Result) FAILURE with reward {reward:.4f}")


        end_time_this_seed = time.time()
        success_rate_this_seed = num_successes / NUM_EPISODES_PER_SEED
        all_success_rates.append(success_rate_this_seed)
        all_detection_times.extend(detection_times_this_seed)
        all_rewards.extend(rewards_this_seed)
        all_episode_lengths.extend(episode_lengths_this_seed)


        print(f"--- Seed {seed} Results ---")
        print(f"Success Rate: {success_rate_this_seed:.4f} ({success_rate_this_seed*100:.2f}%)")
        if detection_times_this_seed:
            print(f"Average Detection Time (Successful Episodes): {np.mean(detection_times_this_seed):.2f}")
        else:
            print("Average Detection Time (Successful Episodes): N/A (no successes)")
        print(f"Average Episode Reward: {np.mean(rewards_this_seed):.4f}")
        print(f"Average Episode Length: {np.mean(episode_lengths_this_seed):.2f}")
        print(f"Time taken for this seed: {end_time_this_seed - start_time_this_seed:.2f} seconds")


    total_experiment_end_time = time.time()

    # --- 3.4. Display Final Results ---
    mean_success_rate = np.mean(all_success_rates)
    std_success_rate = np.std(all_success_rates)
    mean_detection_time = np.mean(all_detection_times) if all_detection_times else -1
    std_detection_time = np.std(all_detection_times) if all_detection_times else -1
    mean_reward = np.mean(all_rewards)
    std_reward = np.std(all_rewards)
    mean_episode_length = np.mean(all_episode_lengths)
    std_episode_length = np.std(all_episode_lengths)


    print(f"\n--- Overall {LOOKAHEAD_K}-step Lookahead Policy Results ---")
    print(f"Total experiment time across {NUM_SEEDS} seeds: {total_experiment_end_time - total_experiment_start_time:.2f} seconds")
    print(f"Average Success Rate over {NUM_SEEDS} seeds and {NUM_EPISODES_PER_SEED} episodes each: {mean_success_rate:.4f} ({mean_success_rate*100:.2f}%)")
    print(f"Standard Deviation of Success Rate: {std_success_rate:.4f}")
    if all_detection_times:
        print(f"Average Detection Time (Successful Episodes) across all seeds: {mean_detection_time:.2f}")
        print(f"Standard Deviation of Detection Time: {std_detection_time:.2f}")
    else:
        print("Average Detection Time (Successful Episodes) across all seeds: N/A (no successes)")

    print(f"Average Episode Reward across all seeds: {mean_reward:.4f}")
    print(f"Standard Deviation of Reward: {std_reward:.4f}")
    print(f"Average Episode Length across all seeds: {mean_episode_length:.2f}")
    print(f"Standard Deviation of Episode Length: {std_episode_length:.2f}")

--- Running Empirical Experiment for k-step lookahead Policy over Multiple Seeds ---
Number of seeds: 10
Number of evaluation episodes per seed: 10
Lookahead depth (k): 1
Rollout simulations: 10
--------------------

--- Running with Seed: 0 ---
--- Seed 0 Results ---
Success Rate: 0.2000 (20.00%)
Average Detection Time (Successful Episodes): 13.00
Average Episode Reward: -2.7656
Average Episode Length: 18.60
Time taken for this seed: 222.41 seconds

--- Running with Seed: 1 ---
--- Seed 1 Results ---
Success Rate: 0.0000 (0.00%)
Average Detection Time (Successful Episodes): N/A (no successes)
Average Episode Reward: -2.5895
Average Episode Length: 20.00
Time taken for this seed: 213.83 seconds

--- Running with Seed: 2 ---
--- Seed 2 Results ---
Success Rate: 0.2000 (20.00%)
Average Detection Time (Successful Episodes): 9.50
Average Episode Reward: -2.7598
Average Episode Length: 17.90
Time taken for this seed: 168.49 seconds

--- Running with Seed: 3 ---
--- Seed 3 Results ---
Succes